## 12. 윤곽선 검출
- 경게선을 연결한 선
- 흑백 이미지 -> 이진화 -> 윤곽선 찾기(findcontours) -> 윤관선 그리기 (drawcontours)

In [2]:
import cv2


DOG = "images/dog.jpg"
VEHICLE = "images/vehicles.png"

In [8]:
img = cv2.imread(DOG)
copied = img.copy()
copied_ext = img.copy()

gray = cv2.cvtColor(copied, cv2.COLOR_BGR2GRAY)

# 이진화
ret, binary = cv2.threshold(gray, -1,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 윤곽선 찾기
contours, hierachy =  cv2.findContours(binary, cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
contours_ext, hierachy_ext =  cv2.findContours(binary, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

# 윤곽선 그리기
cv2.drawContours(copied,contours,-1, (0,255,0), 2)
cv2.drawContours(copied_ext,contours_ext,-1, (0,255,0), 2)

cv2.imshow("img",img)
cv2.imshow("contours",copied)
cv2.imshow("contours_ext",copied_ext)

cv2.waitKey(0)
cv2.destroyAllWindows()

### 12-2. BoundingRect
- 윤곽선을 둘러싼 사각형을 그리기

In [52]:
img = cv2.imread(DOG)
copied = img.copy()


gray = cv2.cvtColor(copied, cv2.COLOR_BGR2GRAY)

# 이진화
ret, binary = cv2.threshold(gray, -1,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 윤곽선 찾기
contours, hierachy =  cv2.findContours(binary, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

for contour in contours:
    x,y,width,height = cv2.boundingRect(contour)
    cv2.rectangle(copied,(x,y),(x+width,y+height),(0,0,255),2)

# 윤곽선 그리기
# cv2.drawContours(copied,contours,-1, (0,255,0), 2)

cv2.imshow("img",img)
cv2.imshow("contours",copied)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 실습 3  순서대로 박스 표시
- 인덱스 트랙바를 만들고
- 트랙바의 인덱스 값을 변경하면
- 생생된 박스가 순서대로 화면에 띄워지도록 구현하기

In [53]:
img = cv2.imread(VEHICLE)
name = "VEHICLE"
copied = img.copy()
cv2.namedWindow(name)
# 이진화 처리
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 윤곽선 찾기
contours, hierachy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.createTrackbar("INDEX",name,0,len(contours) - 1,lambda x:x)

while True:
    index = cv2.getTrackbarPos("INDEX", name)

    copied = img.copy()  
    x, y, width, height = cv2.boundingRect(contours[index])
    cv2.rectangle(copied, (x, y), (x + width, y + height), (0, 0, 255), 2)
    cv2.imshow(name, copied)

    if cv2.waitKey(1) == ord("q"): 
        break

cv2.destroyAllWindows()

### 12-3. contourArea

In [50]:
img = cv2.imread(VEHICLE)
name = "VEHICLE"
copied = img.copy()
cv2.namedWindow(name)
# 이진화 처리
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# 윤곽선 찾기
contours, hierachy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


# 윤곽선의 면적이 특정 크기 이상인 윤곽선을 새로운 리스트에 저장
MIN_AREA = 1000
filtered_counters = []
for contour in contours:
    if cv2.contourArea(contour) > MIN_AREA:
        filtered_counters.append(contour)

cv2.createTrackbar("INDEX",name,0,len(filtered_counters) - 1,lambda x:x)

while True:
    index = cv2.getTrackbarPos("INDEX", name)

    copied = img.copy()  
    x, y, width, height = cv2.boundingRect(filtered_counters[index])
    cv2.rectangle(copied, (x, y), (x + width, y + height), (0, 0, 255), 2)
    cv2.imshow(name, copied)

    if cv2.waitKey(1) == ord("q"): 
        break

cv2.destroyAllWindows()

## 실습 4 카드 하나씩 새 창에 표시

In [42]:
import cv2
import numpy as np

# 이미지 읽기
img = cv2.imread("images/playing_cards.png")  # 이미지 경로 수정
cv2.imshow("Original Image", img)

# 이진화 처리
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)


contours, hierachy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img,contours,-1,(0,0,255),2)


index = 1
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    
    if w > 50 and h > 100:  
        card = img[y:y+h, x:x+w] 
        
        cv2.imshow(f"Card {index}", card)
        index += 1

cv2.waitKey(0) 
cv2.destroyAllWindows()


## 트랙바를 이용한 카드 선택 후 출력

In [48]:
import cv2

# 이미지 읽기
img = cv2.imread("images/playing_cards.png")  # 이미지 경로 수정
if img is None:
    print("이미지를 읽을 수 없습니다. 경로를 확인하세요.")
    exit()

# 원본 이미지 복사본 생성 (사각형 그릴 이미지)
original_with_boxes = img.copy()

# 이진화 처리
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 윤곽선 찾기
contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 윤곽선의 면적이 특정 크기 이상인 윤곽선을 새로운 리스트에 저장
MIN_AREA = 1000
filtered_contours = []
for contour in contours:
    if cv2.contourArea(contour) > MIN_AREA:
        filtered_contours.append(contour)

# 트랙바 이름 및 창 이름 설정
trackbar_name = "INDEX"
window_name = "Filtered Cards"

cv2.namedWindow(window_name)
cv2.createTrackbar(trackbar_name, window_name, 0, len(filtered_contours) - 1, lambda x: x)

while True:
    # 트랙바 값 가져오기
    index = cv2.getTrackbarPos(trackbar_name, window_name)

    # 원본 이미지 복사
    copied = original_with_boxes.copy()

    # 선택된 윤곽선에 대해 사각형 표시 및 카드 추출
    x, y, width, height = cv2.boundingRect(filtered_contours[index])
    cv2.rectangle(copied, (x, y), (x + width, y + height), (0, 0, 255), 2)

    # 선택된 카드 영역 추출
    selected_card = img[y:y + height, x:x + width]

    # 결과 출력
    cv2.imshow(window_name, copied)
    cv2.imshow("Selected Card", selected_card)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) == ord("q"):
        break

cv2.destroyAllWindows()


## 많은 카드

In [49]:
import cv2 as cv
img = cv.imread("images/many_cards.png")
img = cv.resize(img, (1200, 600))
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
ret, otsu = cv.threshold(gray, -1, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
contours, hierachy = cv.findContours(otsu, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
filtered_contours = [] 
for contour in contours:
    if cv.contourArea(contour) > 200:
        filtered_contours.append(contour)
cv.namedWindow('img')
cv.createTrackbar('Index', 'img', 0, len(filtered_contours) - 1, lambda x: x)
window_name = None
previous_index = -1
cv.imshow('img', img)
while True:
    index = cv.getTrackbarPos('Index', 'img')
    copied_img = img.copy()
    x, y, width, height = cv.boundingRect(filtered_contours[index])
    cv.rectangle(copied_img, (x, y), (x + width, y + height),
                 (0, 0, 255), 5, cv.LINE_AA)
    cv.imshow('img', copied_img)
    if index != previous_index:
        x1, y1, width, height = cv.boundingRect(filtered_contours[index])
        x2 = x1 + width
        y2 = y1 + height
        separated_img = img[y1:y2, x1:x2]
        separated_img = cv.resize(separated_img, (150, 200))
        window_name = "CARD"
        cv.imshow(window_name, separated_img)
        previous_index = index
    if cv.waitKey(1) != -1:
        break
cv.destroyAllWindows()